# Car Prices based on Car Features

In this notebook, I will be extracting the prices for different types of cars based on certain features of an automobile (i.e. number of doors, body style, type of gas/engine, etc..)

## The Data

First, let's import the data and models we will need.

You can find the raw data at https://www.openml.org/d/9

The below code takes around ~19 secs to run for me (MacBook Pro (13-inch, 2020, 1.4 GHz Quad-Core Intel Core i5, 16 GB 2133 MHz LPDDR3))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

X, y = fetch_openml("autos", version=1, as_frame=True, return_X_y=True)
data = X
data['target'] = y

#let's take a quick peak at the data
print(data)

So what questions am I looking to answer with the data I now have? Mainly, I'm curious in the average price of a car based on specific features. Moreover, I am curious as to which brands of cars are the most expensive and which are more affordable.

## Generating a Pivot Table

Now, I'll load the data into a dataframe and generate a pivot table with it.

In [ ]:
pivot = np.round(pd.pivot_table(data, values='price', 
                                index='num-of-doors', 
                                columns='fuel-type', 
                                aggfunc=np.mean),2)
print(pivot)

This is a pretty barebones table, but it shows the key info: price for types of cars based on certain features of an automobile and types of gas. Let's get more complicated.

In [ ]:
pivot = np.round(pd.pivot_table(data, values='price', 
                                index=['num-of-doors', 'body-style'], 
                                columns=['fuel-type', 'fuel-system'], 
                                aggfunc=np.mean,
                                fill_value=0),2)
                                
print(pivot)

Okay! Now we have some serious data. We can see the cost of fuel (in cents) for different types of cars with different features for each car.

## Now let's make some pretty plots!

Pandas is awesome and I barely have to do any work to plot the above data...

In [ ]:
np.round(pd.pivot_table(data, values='price', 
                                index=['make'], 
                                columns=['num-of-doors'], 
                                aggfunc=np.mean,
                                fill_value=0),2).plot.barh(figsize=(10,7),
                                                          title='Mean car price by make and number of doors')


By simply adding .plot(), I am able to render this nice histograph of car makers, the number of doors of their cars, and the average price of their cars.

From this, we can see that the most expensive brands (for 4 doors) is mercedes and jaguar and that the most affordable brand is plymouth, isuzu, and chevrolet.

## Pictures and plots are great, but let's work with numbers

I'll start by calculating from meaningful statistics: the mean and median price for car body style and the number of doors.

In [ ]:
np.round(pd.pivot_table(data, values='price', 
                                index=['body-style'], 
                                columns=['num-of-doors'], 
                                aggfunc=[np.mean, np.median],
                                fill_value=0),2)

I think that the destinction between number of doors is important, but it isn't vital. 

In [ ]:
np.round(pd.pivot_table(data, values='price', 
                                index=['body-style'], 
                                columns=['num-of-doors'], 
                                aggfunc=[np.sum],
                                fill_value=0,
                                margins=True, margins_name='Total'),2)

Awesome, now we have the sums of both! Let's finally draw some meaningful insights....

## So what can we learn?

Let's style the data and begin looking at it critically.

In [ ]:
pivot = np.round(pd.pivot_table(data, values=['price', 'horsepower'], 
                                index=['make'], 
                                aggfunc=np.mean,
                                fill_value=0),2)

pivot.style.format({'price':'${0:,.0f}',
                   'horsepower':'{0:,.0f}hp'})

Okay, this is all we need. We have our info and we can draw some pretty good conclusions. For instance, the best 'bang for your buck' brand so-to-speak is Mercury!

I like pictures better than data tables, however, so let's generate one!

In [ ]:
pivot = np.round(pd.pivot_table(data, values=['price', 'horsepower'], 
                                index=['make'], 
                                aggfunc=np.mean,
                                fill_value=0),2)
                                
pivot = pivot.reindex(pivot['price'].sort_values(ascending=False).index).nlargest(10, 'price')

pivot.style.format({'price':'${0:,.0f}',
                   'horsepower':'{0:,.0f}hp'}).bar(color='#d65f5f')

## Using .groupby()

Per the assignment's instructions, using the .groupby() method is necessary to comprehensively understand a data set. Similarly to the above processes, viewing the data grouped by specific features (like in a pivot table) is useful. Above, I grouped by maker and by car features. Below, I will use .groupby() to group cars by their fuel-type.

In [ ]:
data.groupby(['fuel-type']).mean()

So, looking at the above results, we can deduce soe pretty useful average information regarding cars of different fuel types. For exmple, we can see that the average hoursepower of cars using gas is on average higher than that of cars using diesel. 